#Silver Data Transformations

In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, col

In [0]:
  df=spark.readStream.format("delta")\
        .option("header",True)\
        .option("inferSchema",True)\
        .load("abfss://silver@anamikanetflixdl.dfs.core.windows.net/Netflix_directors")
  df.display()

In [0]:
df=spark.read.format('delta').option('header',True).option('inferschema',True)\
.load("abfss://bronze@anamikanetflixdl.dfs.core.windows.net/netflix_titles")
df.display()

In [0]:
df=df.fillna({"duration_minutes":0,"duration_seasons":1})
df.display()

In [0]:
df = df.withColumn("duration_minutes",col('duration_minutes').cast(IntegerType()))\
            .withColumn("duration_seasons",col('duration_seasons').cast(IntegerType()))

In [0]:
df.printSchema()

In [0]:
%python 
df = df.withColumn(
    "type_flag",
    when(col("type").cast("string") == "Movie", 1)
    .when(col("type").cast("string") == "TV Show", 2)
    .otherwise(0)
)
# df = df.withColumn("type_flag",when(col('type')=='Movie',1)\
#                         .when(col('type')=='TV Show',2)\
#                         .otherwise(0))
display(df)

In [0]:
df=df.withColumn("ShortTitle",split(col("title"),":").getItem(0))
df.display()

In [0]:
df=df.withColumn("rating",split(col("rating"),"-")[0])
df.display()

##RANKING DATA ACCORDING TO DURATION MINUTES

In [0]:
# from pyspark.sql.functions import col,split
# from pyspark.sql.types import IntegerType
# df=df.withColumn("duration_minutes",col("duration_minutes").cast(IntegerType()))\
# .withColumn("duration_seasons",col("duration_seasons").cast(IntegerType()))

In [0]:
df= df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col("duration_minutes").desc())))
df.display()

In [0]:
df.createOrReplaceTempView("temp_view")

In [0]:
df.createOrReplaceGlobalTempView("global_view")

In [0]:
df = spark.sql(""" 
                select * from global_temp.global_view
               
               """)

In [0]:
df.display()

Want to count how many movies & TV shows?

In [0]:
from pyspark.sql.functions import count

df_vis = df.groupBy("type").agg(count('*').alias('total_cnt'))
display(df_vis)

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta").mode("overwrite").option("path","abfss://silver@anamikanetflixdl.dfs.core.windows.net/netflix_titles").save()